http://www.local10.com/sports/college-basketball/miami-florida-florida-state-all-heading-to-big-dance

In [185]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import element
from time import sleep
import pandas as pd
import datetime


def get_page_data(year):
    url = 'http://www.sports-reference.com/cbb/postseason/'+str(year)+'-ncaa.html'
    page = urlopen(url)
    return BeautifulSoup(page, 'html.parser')

def tidy_location(loc_str):
    if loc_str.startswith('at '):
        return loc_str[len('at '):]

def tidy_team_name(team_name):
    team_name.replace(';','')

In [186]:
first_year_of_tourney = 1939
def get_bracket():
    # this works for 84 first round bye and 85 and later 64+ team bracket ignoring play-in games
    now = datetime.datetime.now()
    years = range(now.year, first_year_of_tourney - 1, -1)
    bracket = []
    for year in years: 
        soup = get_page_data(year)
        region_tags = soup.find_all("div", attrs={"data-controls": "#brackets"})[0].select('div a')           
        #print(year)
        for region_tag in region_tags:        
            region = str(region_tag.string).lower()                           
            if region == 'final four':
                region_div = soup.find('div', {'id':'national'})
            else:
                search = {'id': region} 
                region_div = soup.find('div',**search)            
            if region_div is not None:
                bracket_rounds = region_div.div
                div_rounds = bracket_rounds.find_all('div', {'class':'round'})                
                if region == 'final four':
                    bracket_round = 5
                else:
                    bracket_round = 1
                for div_round in div_rounds:
                    #print("round, ", bracket_round)
                    if isinstance(div_round, element.Tag ):
                        #print(len(div_round))
                        for gm in div_round:
                            if isinstance(gm, element.Tag ):
                                #print(gm)
                                #print("len contents: ", len(gm.contents))
                                #print()
                                if (len(gm.contents) >= 9):
                                    #print(gm)
                                    #print()
                                    seeds_loc = gm.find_all('span')
                                    if len(seeds_loc) > 2:                                        
                                        winner = gm.find('div', {'class':'winner'})
                                        if winner is not None:
                                            winner_seed = winner.find('span').string                                        
                                        seed1 = seeds_loc[0].string                                        
                                        seed2 = seeds_loc[1].string                                        
                                        currLoc = tidy_location(seeds_loc[2].string)        
                                        alist = gm.select('div a')    
                                        if len(alist) > 2:
                                            #team1 = tidy_team_name(alist[0].string)
                                            team1 = alist[0].string
                                            #if len(alist) > 2:
                                            #team2 = tidy_team_name(alist[2].string)
                                            team2 = alist[2].string
                                            #else:
                                            #    print(alist)
                                            bracketRow = [year, region, bracket_round, seed1, team1, currLoc, seed2, team2, winner_seed]
                                            #print(bracketRow)
                                            bracket.append(bracketRow)
                        bracket_round += 1
        sleep(0.5)
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Round', 'Seed', 'Team', 'Location', 'OpponentSeed', 'Opponent', 'WinningSeed']) 

    
df = get_bracket()
df.to_csv('ncaa.csv')

In [187]:
df1 = df.loc[(df['Round'] == 1)]
fl_list = ['Florida', 'Florida State', 'Miami (FL)', 'UCF', 'Florida Gulf Coast', 'South Florida', 'Florida A&M;','Florida International', 'Florida Atlantic']

df1_teams_year = df1.loc[:, ['Year','Team','Opponent']]
df1_teams = df1.loc[:, ['Team','Opponent']]

fl_team_app = []
for fl_tm in fl_list:
    fl_tm_count = df1_teams[(df1_teams.Team == fl_tm) | (df1_teams.Opponent == fl_tm)]
    fl_team_app.append([fl_tm, len(fl_tm_count.index)])
sorted(fl_team_app, key=lambda l:l[1], reverse=True)

[['Florida', 19],
 ['Florida State', 15],
 ['Miami (FL)', 9],
 ['UCF', 3],
 ['Florida Gulf Coast', 3],
 ['South Florida', 3],
 ['Florida A&M;', 1],
 ['Florida International', 1],
 ['Florida Atlantic', 1]]

In [188]:
team_counts= df1_teams.stack().value_counts()

print(team_counts)
team_counts.to_csv('ncaa_team_appearances.csv')

Kansas                       38
UNC                          38
Kentucky                     37
Duke                         35
Villanova                    35
Syracuse                     34
Arizona                      34
Notre Dame                   33
Louisville                   33
UCLA                         32
UConn                        32
Temple                       31
Marquette                    31
Texas                        31
Indiana                      30
Oklahoma                     28
Michigan State               27
Illinois                     27
West Virginia                27
St. John's (NY)              27
Purdue                       27
Georgetown                   26
Arkansas                     26
Xavier                       25
Pitt                         25
Princeton                    25
Cincinnati                   24
Maryland                     24
Missouri                     24
Oklahoma State               24
                             ..
Binghamt

In [229]:
fl_teams_by_year = df1_teams_year[df1_teams_year.Team.isin(fl_list)|df1_teams_year.Opponent.isin(fl_list)] 
fl_teams_by_year
fl_teams_by_year.to_csv('fl_teams_by_year.csv')

In [207]:
df1_teams_year_t = df1_teams_year.loc[:, ['Year','Team']][df1_teams_year.Team.isin(fl_list)]
df1_teams_year_o = df1_teams_year.loc[:, ['Year','Opponent']][df1_teams_year.Opponent.isin(fl_list)]
df1_teams_year_o.rename(columns={'Opponent':'Team'}, inplace=True)
df1_teams_fl_agg = pd.concat([df1_teams_year_t, df1_teams_year_o])
df1_teams_fl_agg.groupby('Year')
final_count = df1_teams_fl_agg.groupby('Year').count().sort_index(axis=0, ascending=False)
final_count


,Team
Year,
2017,4
2016,2
2014,1
2013,3
2012,3
2011,2
2010,2
2009,1
2008,1


In [228]:

df1_teams_fl_agg2 = df1_teams_fl_agg.groupby('Year')
for year, fl_group in df1_teams_fl_agg2:
    print(fl_group.to_string(index=False))
    print()
    

Year        Team
1960  Miami (FL)

Year           Team
1968  Florida State

Year           Team
1972  Florida State

Year           Team
1978  Florida State

Year           Team
1980  Florida State

Year     Team
1987  Florida

Year           Team
1988        Florida
1988  Florida State

Year           Team
1989        Florida
1989  Florida State

Year           Team
1990  South Florida

Year           Team
1991  Florida State

Year           Team
1992  Florida State
1992  South Florida

Year           Team
1993  Florida State

Year     Team
1994  Florida
1994      UCF

Year                   Team
1995                Florida
1995  Florida International

Year Team
1996  UCF

Year           Team
1998  Florida State
1998     Miami (FL)

Year          Team
1999    Miami (FL)
1999       Florida
1999  Florida A&M;

Year        Team
2000     Florida
2000  Miami (FL)

Year     Team
2001  Florida

Year              Team
2002           Florida
2002        Miami (FL)
2002  Florida Atlantic

Year 